In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import scipy as sc
from sklearn import preprocessing
from sklearn import model_selection
import functools

from recsys.preprocess import *
from recsys.utility import *

RANDOM_STATE = 2342

%matplotlib inline

In [2]:
train = pd.read_csv('data/train_final.csv', delimiter='\t')
playlists = pd.read_csv('data/playlists_final.csv', delimiter='\t')
tracks = pd.read_csv('data/tracks_final.csv', delimiter='\t')
tracks = tracks[tracks['duration'] != -1]

target_playlists = pd.read_csv('data/target_playlists.csv', delimiter='\t')
target_tracks = pd.read_csv('data/target_tracks.csv', delimiter = '\t')

In [ ]:
len(train)

In [ ]:
tracks.head()

In [3]:
playlists.head()

,created_at,playlist_id,title,numtracks,duration,owner
0,1216545588,644838,[12727],27,6522,41504
1,1249326867,7577564,[],9,2650,41504
2,1257766688,3120683,[183],16,3645,44542
3,1248079275,4278112,"[12389, 18698, 18925, 11695, 7117]",15,4151,44542
4,1175201268,8656823,"[12809, 2095, 13257, 12671, 20426, 14448, 18698]",84,18414,44542


In [28]:
len(playlists)

57561

In [ ]:
train.head()

In [4]:
train, test, target_playlists, target_tracks = train_test_split(train, test_size=0.3, min_playlist_tracks=7)

In [ ]:
train.head()

In [ ]:
len(train)

In [ ]:
test.head()

In [ ]:
len(test)

In [27]:
target_playlists

,playlist_id,track_ids
playlist_id,,
10225531,10225531,[]
3676000,3676000,[]
5221968,5221968,[]
5678711,5678711,[]
10173471,10173471,[]
2776979,2776979,[]
1105792,1105792,[]
885632,885632,[]
8428894,8428894,[]


In [24]:
len(target_playlists)

8493

In [ ]:
target_tracks

In [ ]:
len(target_tracks)

## Most popular

In [ ]:
# most popular tracks: the ones we want to recommend
most_popular = get_most_popular_tracks(train)

# we don't want to recommend tracks already in the playlists...
tracks_in_playlist = get_playlist_track_list2(train)

In [ ]:
tracks_in_playlist

In [ ]:
tracks_in_playlist.loc[3271849]['track_ids']

In [ ]:
most_popular

In [ ]:
tracks_to_suggest = most_popular.index.values

In [ ]:
tracks_to_suggest

In [ ]:
target_playlists

In [ ]:
predictions = pd.DataFrame(target_playlists)
predictions.index = target_playlists['playlist_id']
predictions['track_ids'] = [np.array([]) for i in range(len(predictions))]

In [ ]:
predictions

In [ ]:
"""for pl_id in target_playlists['playlist_id']:
    count = 0
    i = 0
    pred = []
    while count < 5:
        if tracks_to_suggest[i] not in tracks_in_playlist.loc[pl_id]['track_ids']:
            # Predict track i
            # IMPORTANT: should we check if the track to suggest is in target_tracks?
            pred.append(tracks_to_suggest[i])
            count += 1
        i += 1
    predictions[pl_id] = pred
    """
for it,row in target_playlists.iterrows():
    count = 0
    i = 0
    pred = []
    while count < 5:
        if tracks_to_suggest[i] not in tracks_in_playlist.loc[row['playlist_id']]['track_ids']:
            # Predict track i
            # IMPORTANT: should we check if the track to suggest is in target_tracks?
            pred.append(tracks_to_suggest[i])
            count += 1
        i += 1
    predictions.loc[row['playlist_id']] = predictions.loc[row['playlist_id']].set_value('track_ids', np.array(pred))

In [ ]:
predictions

In [ ]:
evaluate(test, predictions)

## Content based

In [ ]:
tracks

In [5]:
tracks['tags'] = tracks['tags'].apply(lambda x: np.array(eval(x)))

In [ ]:
tracks

In [6]:
counter = 0
def reduceCount(prev, l):
    global counter
    for el in l:
        if el not in prev:
            prev[el] = [counter]
        else:
            prev[el] += [counter]
    counter += 1
    return prev

# key: tag_id, value: [track_idx...]
distinct_tags = functools.reduce(reduceCount, tracks['tags'], dict())

In [ ]:
distinct_tags[54087]

In [ ]:
len(distinct_tags)

In [ ]:
len(tracks)

In [7]:
most_popular_tags = [k for k,v in sorted([(k, len(v)) for k, v in distinct_tags.items()], key=lambda tup: tup[1], reverse=True)]
most_popular_tags

[205245,
 115355,
 70618,
 11056,
 81223,
 189631,
 154891,
 46208,
 11242,
 115752,
 43212,
 11957,
 218701,
 3982,
 117167,
 122769,
 84597,
 54087,
 103394,
 35060,
 226723,
 76913,
 70625,
 3668,
 4425,
 107401,
 6120,
 195173,
 193464,
 105199,
 31015,
 64978,
 3424,
 227682,
 255208,
 186445,
 107398,
 116047,
 170251,
 194264,
 50604,
 115684,
 198998,
 177424,
 195456,
 70251,
 153432,
 89467,
 204710,
 24358,
 144067,
 92799,
 144192,
 97480,
 191177,
 191251,
 31253,
 201327,
 100728,
 50247,
 57528,
 103055,
 237214,
 193395,
 97598,
 109806,
 169455,
 35233,
 72354,
 87197,
 194413,
 239459,
 45739,
 83064,
 254186,
 235879,
 154941,
 236955,
 38403,
 112283,
 249989,
 191642,
 100238,
 42894,
 271235,
 216938,
 153731,
 157349,
 136980,
 175331,
 50764,
 122106,
 60046,
 68687,
 68498,
 3838,
 64267,
 55854,
 29723,
 116155,
 190991,
 61837,
 97621,
 66729,
 81211,
 11191,
 215342,
 80989,
 65509,
 71028,
 190049,
 84911,
 189007,
 190112,
 91383,
 219398,
 33826,
 212982,

In [8]:
from scipy.sparse import *
BEST_TAGS = 30
most_popular_tags_best = most_popular_tags[0:BEST_TAGS]

In [ ]:
most_popular_tags_best

In [ ]:
ICM = lil_matrix((BEST_TAGS,len(tracks)))
for tag_it, tag_id in enumerate(most_popular_tags_best):
    l = distinct_tags[tag_id]
    for track_idx in l:
        ICM[tag_it,track_idx] = 1
        
# ICM["index of tag in most_popular_tags_best", "index of track in tracks"]

In [ ]:
ICM

In [9]:
tracks = get_track_tags_binary(tracks, cut_off=BEST_TAGS, relevant_tags=most_popular_tags_best)

In [10]:
tracks

,track_id,artist_id,duration,playcount,album,tags,binary_tags
0,2972914,144,224000,49.0,[7],"[54087, 1757, 1718, 116712, 189631]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2750239,246,157000,1.0,[8],"[189631, 3424, 177424, 46208, 205245]","[1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,1550729,144,217000,554.0,[9],"[54087, 109806, 46869, 183258, 54337]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,2169950,144,207000,200.0,[9],"[54087, 70618, 207003, 109806, 116712]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,1903709,144,198000,5.0,[None],"[54087, 81223, 116712, 215342, 71028]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,2256817,144,218000,2.0,[9],"[54087, 109806, 189631, 49166, 116712]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,2561768,928,223000,249.0,[26],"[50764, 4425, 11056, 205245, 81223]","[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,474864,928,193000,73.0,[22],"[205245, 81223, 11056, 267, 3982]","[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
8,1378455,928,304000,73.0,[22],"[11056, 205245, 81223, 189631, 84597]","[1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
9,1523190,928,206000,10.0,[22],"[205245, 11056, 81223, 4425, 189631]","[1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [11]:
bt = [ar.tolist() for ar in tracks['binary_tags'].values]
bt

[[0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  0,
  0,
  1,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  0,
  0,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [12]:
%%time
from sklearn.neighbors import NearestNeighbors

X = np.array(bt)#np.array([[0, 1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree', n_jobs=-1).fit(X)

CPU times: user 36.9 s, sys: 293 ms, total: 37.2 s
Wall time: 37.2 s


In [13]:
distances, indices = nbrs.kneighbors(np.array([tracks['binary_tags'][0].tolist()]))
indices

array([[1913, 1915, 4828,    0, 2189]])

In [14]:
a = tracks['binary_tags']
a = a.apply(lambda l: l.tolist())

In [15]:
a = a.tolist()

In [18]:
b = a

In [19]:
%%time
r1,r2 = nbrs.kneighbors(np.array(b))

CPU times: user 13min 8s, sys: 1.77 s, total: 13min 9s
Wall time: 3min 29s


In [20]:
r1

array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [26]:
r2

array([[ 1913,  1915,  4828,     0,  2189],
       [72253, 72586, 72585, 52609, 72584],
       [   53,   185,   183,     2,   105],
       ..., 
       [23495, 52425, 71074, 97210, 70995],
       [23495, 52425, 71074, 97210, 70995],
       [23495, 52425, 71074, 97210, 70995]])

In [ ]:
S = lil_matrix((len(target,len(tracks)))
for tag_it, tag_id in enumerate(most_popular_tags_best):
    l = distinct_tags[tag_id]
    for track_idx in l:
        ICM[tag_it,track_idx] = 1
        
# ICM["index of tag in most_popular_tags_best", "index of track in tracks"]

In [22]:
predictions = pd.DataFrame(target_playlists)
predictions.index = target_playlists['playlist_id']
predictions['track_ids'] = [np.array([]) for i in range(len(predictions))]

In [23]:
predictions

,playlist_id,track_ids
playlist_id,,
10225531,10225531,[]
3676000,3676000,[]
5221968,5221968,[]
5678711,5678711,[]
10173471,10173471,[]
2776979,2776979,[]
1105792,1105792,[]
885632,885632,[]
8428894,8428894,[]


In [ ]:
for it,row in target_playlists.iterrows():
    count = 0
    i = 0
    pred = []
    while count < 5:
        
        if tracks_to_suggest[i] not in tracks_in_playlist.loc[row['playlist_id']]['track_ids']:
            # Predict track i
            # IMPORTANT: should we check if the track to suggest is in target_tracks?
            pred.append(tracks_to_suggest[i])
            count += 1
        i += 1
    predictions.loc[row['playlist_id']] = predictions.loc[row['playlist_id']].set_value('track_ids', np.array(pred)

In [ ]:
evaluate(test, predictions)